In [2]:
import findspark
findspark.init()

import pyspark
from pyspark import SparkContext

In [3]:
from pyspark.sql import SparkSession

In [4]:
spark = SparkSession.builder.appName('Read CSV Write Parquet').getOrCreate()

In [5]:
df = spark.read.format("csv").load("input_files/*.csv")

In [6]:
df.show(5)

+--------------------+--------------------+--------------------+--------+---+-------+-------+---+------+
|                 _c0|                 _c1|                 _c2|     _c3|_c4|    _c5|    _c6|_c7|   _c8|
+--------------------+--------------------+--------------------+--------+---+-------+-------+---+------+
|8e66dea62a914ba69...|2019-09-12T00:00:...|http://example.co...|user-028| ER| Chrome|    Mac|501|0.6975|
|3897a5a7115d489e9...|2019-09-12T00:00:...|http://example.co...|user-032| SJ|Firefox|  Linux|307|0.3905|
|baf865e583aa45f79...|2019-09-12T00:01:...|http://example.co...|user-031| MA|Firefox|  Linux|510|0.5118|
|656bd941b94044a18...|2019-09-12T00:01:...|http://example.co...|user-057| GD|   Edge|  Linux|510|0.1552|
|a4156ae1faf74acc9...|2019-09-12T00:02:...|http://example.co...|user-054| ZW|   Edge|Android|208|0.7311|
+--------------------+--------------------+--------------------+--------+---+-------+-------+---+------+
only showing top 5 rows



In [9]:
#df.write.parquet("hw5_problem2.parquet")

In [127]:
df = spark.read.load("hw5_problem2.parquet")

In [128]:
df.show(5)

+--------------------+--------------------+--------------------+--------+---+-------+-------+---+------+
|                 _c0|                 _c1|                 _c2|     _c3|_c4|    _c5|    _c6|_c7|   _c8|
+--------------------+--------------------+--------------------+--------+---+-------+-------+---+------+
|eefc2eb34a354ab7a...|2019-09-12T00:00:...|http://example.co...|user-041| CR|     IE|Android|201|0.2856|
|9ba55c683ddd4a05a...|2019-09-12T00:00:...|http://example.co...|user-043| NE| Safari|windows|403|0.2843|
|a45df30287454e598...|2019-09-12T00:01:...|http://example.co...|user-005| VI|   Edge|Android|444|0.6983|
|941c2d880242431c9...|2019-09-12T00:01:...|http://example.co...|user-092| IS|Firefox|windows|226|0.2636|
|97723d7e1669450eb...|2019-09-12T00:02:...|http://example.co...|user-024| GM|Firefox|windows|303|0.0459|
+--------------------+--------------------+--------------------+--------+---+-------+-------+---+------+
only showing top 5 rows



In [195]:
from pyspark.sql.functions import unix_timestamp
from pyspark.sql.functions import from_unixtime
from pyspark.sql.functions import col
from pyspark.sql.functions import lit
from pyspark.sql.functions import date_trunc
from pyspark.sql.functions import count

In [234]:
from pyspark.sql.functions import *

df = spark.read.load("hw5_problem2.parquet")

# reduce df by urls, country, hour
dfReduced = df.select(from_unixtime(unix_timestamp('_c1', "yyyy-MM-dd'T'HH:mm:ss")).alias('datetime')\
                          ,col('_c4').alias('country')\
                          ,col('_c2').alias('url')\
                         )

# specify datetime range
t1 = '2019-09-13 17:00:00'
t2 = '2019-09-14 09:00:00'

# reduce by datetime range
dfReduced = dfReduced.filter(dfReduced["datetime"] >= lit(t1))
dfReduced = dfReduced.filter(dfReduced["datetime"] <= lit(t2))

# add datehour
#dfReduced = dfReduced.withColumn(  'datehour' , date_trunc("hour", "datetime")  )
dfReduced = dfReduced.withColumn(  'datehour' , date_format(col("datetime"), "yyyy-MM-dd:HH") )

dfQueryFour = dfReduced.select('datehour','country','url').drop_duplicates()

In [235]:
#dfQueryFour.orderBy('datehour', ascending=True).show(3)

In [236]:
dfQueryFour.groupBy("datehour", "country").agg(count("*")).orderBy('datehour','country', ascending=True).show(5)

+-------------+-------+--------+
|     datehour|country|count(1)|
+-------------+-------+--------+
|2019-09-13:17|     AD|       2|
|2019-09-13:17|     AE|       6|
|2019-09-13:17|     AF|       1|
|2019-09-13:17|     AI|       2|
|2019-09-13:17|     AL|       1|
+-------------+-------+--------+
only showing top 5 rows



In [ ]:
from pyspark.sql.functions import date_format

df.withColumn("dateColumn",  date_format(col("vacationdate"), "yyyy-MM-dd"))